# Ch5. CNN

#### 패키지 import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

#### CUDA 사용 가능여부

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


#### 에폭, 배치사이즈

In [3]:
EPOCH = 40
BATCH_SIZE = 64

## 데이터 불러오기 : Fashion MNIST

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)


Extracting ./.data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./.data\FashionMNIST\raw



Extracting ./.data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./.data\FashionMNIST\raw



Extracting ./.data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\FashionMNIST\raw



Extracting ./.data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\FashionMNIST\raw
Processing...
Done!


C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#### CNN 클래스

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        
        self.drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = x.view(-1, 320)
        
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x= self.fc2(x)
        
        return x

#### Model과 Optimizer 설정

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## 학습

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch, batch_idx * len(data),
                                                                         len(train_loader.dataset),
                                                                         100. * batch_idx / len(train_loader),
                                                                         loss.item()))

## 평가

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                        reduction='sum').item()
            
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        
        return test_loss, test_accuracy

In [13]:
for epoch in range(1, EPOCH + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:0.847991
Train Epoch: 1 [12800/60000 (21%)]	Loss:0.549969
Train Epoch: 1 [25600/60000 (43%)]	Loss:0.777340
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.524913
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.635507
[1] Test Loss: 0.5327, Accuracy: 79.13%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.909547
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.573917
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.865054
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.576435
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.514963
[2] Test Loss: 0.5117, Accuracy: 80.14%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.633918
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.648740
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.249874
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.444799
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.530011
[3] Test Loss: 0.4461, Accuracy: 83.98%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.678140
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.536541
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.459108
Train Epoch: 4 [38

## ResNet

#### 에폭, 배치사이즈

In [14]:
EPOCHS = 300
BATCH_SIZE = 128

## 데이터 불러오기 : Cifar-10

In [16]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.RandomCrop(32, padding=4),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                         train=False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)


Extracting ./.data\cifar-10-python.tar.gz to ./.data


#### BasicBlock 클래스

In [17]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride !=1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                         kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

#### RestNet 클래스

In [26]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                             stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

#### Model과 Optimizer 설정

In [27]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                     momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50,
                                     gamma=0.1)

In [28]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

## 학습

In [29]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch, batch_idx * len(data),
                                                                         len(train_loader.dataset),
                                                                         100. * batch_idx / len(train_loader),
                                                                         loss.item()))

## 평가

In [30]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                        reduction='sum').item()
            
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        
        return test_loss, test_accuracy

In [33]:
from tqdm import tqdm

In [34]:
for epoch in tqdm(range(1, EPOCHS + 1)):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

Train Epoch: 1 [0/50000 (0%)]	Loss:0.603099
Train Epoch: 1 [25600/50000 (51%)]	Loss:0.483212


  0%|▎                                                                               | 1/300 [00:16<1:21:50, 16.42s/it]

[1] Test Loss: 0.7438, Accuracy: 74.55%
Train Epoch: 2 [0/50000 (0%)]	Loss:0.472833
Train Epoch: 2 [25600/50000 (51%)]	Loss:0.466262


  1%|▌                                                                               | 2/300 [00:33<1:22:19, 16.58s/it]

...

[298] Test Loss: 0.3020, Accuracy: 91.04%
Train Epoch: 299 [0/50000 (0%)]	Loss:0.057658
Train Epoch: 299 [25600/50000 (51%)]	Loss:0.044807


100%|█████████████████████████████████████████████████████████████████████████████▋| 299/300 [1:23:06<00:16, 16.32s/it]

[299] Test Loss: 0.3040, Accuracy: 91.03%
Train Epoch: 300 [0/50000 (0%)]	Loss:0.025008
Train Epoch: 300 [25600/50000 (51%)]	Loss:0.053437


100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [1:23:23<00:00, 16.68s/it]

[300] Test Loss: 0.3032, Accuracy: 91.03%
